In [1]:
# Huggingface
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# Langchain
from langchain.llms import HuggingFacePipeline
from langchain.retrievers import ContextualCompressionRetriever, EnsembleRetriever, ContextualCompressionRetriever, ParentDocumentRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore

# Langchain Huggingface
from langchain_huggingface import HuggingFaceEmbeddings

# Langchain Community
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss

# Langchain Core
from langchain_core.output_parsers import StrOutputParser

# Langchain Teddynote
from langchain_teddynote.retrievers import KiwiBM25Retriever

# Other
import torch
import numpy as np
from numpy.linalg import norm
import os
import fitz
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from langchain_core.documents import Document

/home/user/yhkim/env/RAG_yhkim/lib/python3.10/site-packages/langchain_teddynote/retrievers/__init__.py:14: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from .kiwi_bm25 import KiwiBM25Retriever


In [2]:
def l2_distance(vec1, vec2):
    return np.sqrt(np.sum((vec1 - vec2) ** 2))

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))       #norm: element wise suqare sum and root

def dot_product(vec1, vec2):
    return np.dot(vec1, vec2)

# Load PDF

In [3]:
##original code: https://dacon.io/competitions/official/236295/codeshare/11616?page=1&dtype=recent
path = "/home/user/yhkim/data/train_source"         #pdf 파일이 있는 폴더 경로

pdfs = [os.path.join(path, i) for i in os.listdir(path)]


doc_texts = []
for pdf_path in pdfs:
    doc = fitz.open(pdf_path)
    for page in doc:
        text = page.get_text()
    doc_texts.append(
        Document(
            page_content=text,
            metadata={"source": pdf_path},
        ))
        
        
        # {pdf_path: text})

In [7]:
test_texts = [
    Document(
        page_content="Python is a versatile programming language used for web development, data science, and more.",
        metadata={"source": "Introduction to Python"}
    ),
    Document(
        page_content="The Eiffel Tower is one of the most recognizable structures in the world, located in Paris, France.",
        metadata={"source": "Facts about Eiffel Tower"}
    ),
    Document(
        page_content="Machine Learning enables computers to learn and make decisions from data without being explicitly programmed.",
        metadata={"source": "Basics of Machine Learning"}
    ),
    Document(
        page_content="LangChain is a framework for developing applications powered by language models.",
        metadata={"source": "LangChain Overview"}
    ),
    Document(
        page_content="The mitochondria are known as the powerhouse of the cell, providing energy for cellular activities.",
        metadata={"source": "Biology Notes"}
    )
]

doc_texts += test_texts

In [13]:
print(doc_texts[0].page_content[:30])
print()
print(doc_texts[-1].page_content[:30])

- 4 -
6. 주요내용 
①사업규모
- 총사업비 : 

The mitochondria are known as 


# Chunking

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", model_kwargs={"device": "cuda"})
dimension_size = len(embeddings.embed_query(""))

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800, chunk_overlap=50
)
doc_texts = text_splitter.create_documents(texts=[i.page_content for i in doc_texts], metadatas=[i.metadata for i in doc_texts])

## Using faiss index

In [54]:
vectorstore = FAISS(
    embedding_function=embeddings,
    index=faiss.IndexFlatL2(dimension_size),
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [55]:
for i in doc_texts:
    await vectorstore.aadd_documents([i])

In [56]:
print(vectorstore.index.ntotal)
vectorstore.similarity_search_with_score("여러 pdf에서 결산사항들만 모아 보여줘", k=5)

47


[(Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/산업통상자원부_에너지바우처.pdf'}, page_content='비용 부담 완화를 위해 ‘22년 2차추경을 통해 91,617백만원을 증액하여 에너지바우처 지원\n대상을확대하고 지원단가를 인상\n2023\n- 예비비 편성 사유 : 취약계층 난방비 지원 특별대책(’23.1.26, 2.15.)에 따른 취약계층 에너지비용'),
  4.906433),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/국토교통부_행복주택출자.pdf'}, page_content='528,783\n- 건설규모 : ‘24년 2.2만호 준공 등 계속사업 추진\n②사업추진체계\n- 사업시행방법 : 출자, 보고\n- 사업시행주체 : 한국토지주택공사(LH), 지자체(지방공사)\n- 사업 수혜자 : 대\n학\n생\n・사\n회\n초\n년\n생・\n신혼부\n부 등\n 젊\n은층(80%), 고\n령자 및 주\n거취약\n계층\n(20%)'),
  5.00669),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/보건복지부_부모급여(영아수당) 지원.pdf'}, page_content='7. 사업 집행절차\n보건복지부\n지원기준 마련 및 국고보조금 교부\n↓\n시·도\n국고보조금에 지방비(시·도비)를 가산하여 시·군·구에 보조금 교부\n↓\n시·군·구\n지원대상자 선정, 급여액 확정 및 지급 및 정산\n※ e-지로시스템(금융결제원) 또는 각 시‧군‧구 금고 시스템을 통해 지급'),
  6.0614605),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.pdf'}, p

In [57]:
vectorstore = FAISS(
    embedding_function=embeddings,
    index=faiss.IndexFlatL2(dimension_size),
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)
for i in doc_texts:
    await vectorstore.aadd_documents([i])
    
print(vectorstore.index.ntotal)
vectorstore.similarity_search_with_score("아이들의 지원금에 관한 문서를 찾아줘", k=10)

47


[(Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/산업통상자원부_에너지바우처.pdf'}, page_content='비용 부담 완화를 위해 ‘22년 2차추경을 통해 91,617백만원을 증액하여 에너지바우처 지원\n대상을확대하고 지원단가를 인상\n2023\n- 예비비 편성 사유 : 취약계층 난방비 지원 특별대책(’23.1.26, 2.15.)에 따른 취약계층 에너지비용'),
  3.482029),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/보건복지부_부모급여(영아수당) 지원.pdf'}, page_content='7. 사업 집행절차\n보건복지부\n지원기준 마련 및 국고보조금 교부\n↓\n시·도\n국고보조금에 지방비(시·도비)를 가산하여 시·군·구에 보조금 교부\n↓\n시·군·구\n지원대상자 선정, 급여액 확정 및 지급 및 정산\n※ e-지로시스템(금융결제원) 또는 각 시‧군‧구 금고 시스템을 통해 지급'),
  4.838092),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/국토교통부_행복주택출자.pdf'}, page_content='528,783\n- 건설규모 : ‘24년 2.2만호 준공 등 계속사업 추진\n②사업추진체계\n- 사업시행방법 : 출자, 보고\n- 사업시행주체 : 한국토지주택공사(LH), 지자체(지방공사)\n- 사업 수혜자 : 대\n학\n생\n・사\n회\n초\n년\n생・\n신혼부\n부 등\n 젊\n은층(80%), 고\n령자 및 주\n거취약\n계층\n(20%)'),
  4.9144745),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/산업통상자원부_에너지바우처.pdf'}, page_content='부담 완화를 위해

In [59]:
vectorstore = FAISS(
    embedding_function=embeddings,
    index=faiss.IndexFlatIP(dimension_size),
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)
for i in doc_texts:
    await vectorstore.aadd_documents([i])
    
print(vectorstore.index.ntotal)
vectorstore.similarity_search_with_score("아이들의 지원금에 관한 문서를 찾아줘", k=10)

47


[(Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.pdf'}, page_content='허등용.(2018). \x03\n보통교부세의 재정조정기능 분석과 재정분권정책에 주는 시사점. 한국지방세연구원 기본과\n제보고서 2018-06.\n홍근석·김봉균·임정빈.(2021). \x03\n우리나라의 정부 간 재정관계 분석: 자율성과 책임성을 중심으로. 한국정책과\n학학회보, 25(2): 147-173. \n홍근석·여효성(2021). \x03'),
  9.191544),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf'}, page_content='__. (2021). 2021회계연도 중앙관서별 재정사업 자율평가 보고서.\n박정수. (2020). dBrain 기반 재정성과관리 지원 강화 방안. 한국재정정보원 재정정보분석 20-03.\n박노욱 외. (2018). 재정성과평가제도 환류방안에 관한 연구. 한국조세재정연구원 연구보고서 18-14.'),
  8.838391),
 (Document(metadata={'source': '/home/yhkim/data/pdf_practice/test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf'}, page_content='_________. (2022). 「제11회 재정운용전략위원회 개최」.\n_________. (2022). 중기 재정사업 성과관리 추진방향 확정.\n_________. (2021). 「제2회 재정운용전략위원회 개최」.\n대한민국 정부. (2021). 국가재정법 일부개정법률안(대안), 2113611호.\n__________'),
  8.430627),
 (Document(metadata={'source

## Using from_documents method

In [29]:
db = FAISS.from_documents(doc_texts, embedding=embeddings)

db.similarity_search_with_score("Give me docs about stats", k=10)

[(Document(metadata={'source': '/home/user/yhkim/data/train_source/2024년도 성과계획서(총괄편).pdf'}, page_content='전문적 조사ㆍ연구 사업을 통해 과거사 문제 해결에 기여한다.\n3,429\n진실화해위원\n회운영\n진화위운영\n일반회계\n1000-1031\n3,429\n총 계\n3,429'),
  19.619198),
 (Document(metadata={'source': '/home/user/yhkim/data/train_source/재정통계해설.pdf'}, page_content='[04637] 서울특별시 중구 퇴계로 10, 메트로타워   \nT 02-6908-8200   F 02-6312-8959   H www.kpfis.or.kr\nA Guide to Fiscal Statistics\n재정통계 해설'),
  20.681877),
 (Document(metadata={'source': '/home/user/yhkim/data/train_source/1-1 2024 주요 재정통계 1권.pdf'}, page_content='자료: OECD(2024년 3월 추출)\n[표 3-4] 주요국 1인당 부채 추이\n(단위: 일반정부 기준, 달러, %)\n2013\n2014\n2015\n2016\n2017\n2018\n2019\n2020\n2021\n2022\n연평균 \n증가율1)\n미국\n55,483 57,363 59,372 61,665 63,238 67,043 70,298 84,681 88,617 92,513'),
  21.207474),
 (Document(metadata={'source': '/home/user/yhkim/data/train_source/보건복지부_노인일자리 및 사회활동지원.pdf'}, page_content='사업수행기관\n○사업평가\n- 추진사업에대한종합분석및점검\n- 차기년도사업계획수립시반영하여발전방안모색'),
  21.703793),
 (Document(metadata

In [30]:
db.distance_strategy

<DistanceStrategy.EUCLIDEAN_DISTANCE: 'EUCLIDEAN_DISTANCE'>

In [47]:
db = FAISS.from_documents(doc_texts, embedding=embeddings, distance_strategy = DistanceStrategy.MAX_INNER_PRODUCT, normalize_L2 = True)

db.similarity_search_with_score("Give me docs about stats", k=10)

[(Document(metadata={'source': '/home/user/yhkim/data/train_source/재정통계해설.pdf'}, page_content='[04637] 서울특별시 중구 퇴계로 10, 메트로타워   \nT 02-6908-8200   F 02-6312-8959   H www.kpfis.or.kr\nA Guide to Fiscal Statistics\n재정통계 해설'),
  0.44968757),
 (Document(metadata={'source': '/home/user/yhkim/data/train_source/1-1 2024 주요 재정통계 1권.pdf'}, page_content='자료: OECD(2024년 3월 추출)\n[표 3-4] 주요국 1인당 부채 추이\n(단위: 일반정부 기준, 달러, %)\n2013\n2014\n2015\n2016\n2017\n2018\n2019\n2020\n2021\n2022\n연평균 \n증가율1)\n미국\n55,483 57,363 59,372 61,665 63,238 67,043 70,298 84,681 88,617 92,513'),
  0.42388523),
 (Document(metadata={'source': '/home/user/yhkim/data/train_source/2024년도 성과계획서(총괄편).pdf'}, page_content='전문적 조사ㆍ연구 사업을 통해 과거사 문제 해결에 기여한다.\n3,429\n진실화해위원\n회운영\n진화위운영\n일반회계\n1000-1031\n3,429\n총 계\n3,429'),
  0.41133988),
 (Document(metadata={'source': '/home/user/yhkim/data/train_source/1-1 2024 주요 재정통계 1권.pdf'}, page_content='-\n7.1\nOECD 평균\n43,616 46,099 47,431 49,169 50,195 51,835 54,701 63,824 66,674 65,858 \n4.

In [48]:
db.similarity_search_with_score_by_vector(embeddings.embed_query("Give me docs about stats"), k=10, normalize_L2 = True)

[(Document(metadata={'source': '/home/user/yhkim/data/train_source/재정통계해설.pdf'}, page_content='[04637] 서울특별시 중구 퇴계로 10, 메트로타워   \nT 02-6908-8200   F 02-6312-8959   H www.kpfis.or.kr\nA Guide to Fiscal Statistics\n재정통계 해설'),
  0.44968757),
 (Document(metadata={'source': '/home/user/yhkim/data/train_source/1-1 2024 주요 재정통계 1권.pdf'}, page_content='자료: OECD(2024년 3월 추출)\n[표 3-4] 주요국 1인당 부채 추이\n(단위: 일반정부 기준, 달러, %)\n2013\n2014\n2015\n2016\n2017\n2018\n2019\n2020\n2021\n2022\n연평균 \n증가율1)\n미국\n55,483 57,363 59,372 61,665 63,238 67,043 70,298 84,681 88,617 92,513'),
  0.42388523),
 (Document(metadata={'source': '/home/user/yhkim/data/train_source/2024년도 성과계획서(총괄편).pdf'}, page_content='전문적 조사ㆍ연구 사업을 통해 과거사 문제 해결에 기여한다.\n3,429\n진실화해위원\n회운영\n진화위운영\n일반회계\n1000-1031\n3,429\n총 계\n3,429'),
  0.41133988),
 (Document(metadata={'source': '/home/user/yhkim/data/train_source/1-1 2024 주요 재정통계 1권.pdf'}, page_content='-\n7.1\nOECD 평균\n43,616 46,099 47,431 49,169 50,195 51,835 54,701 63,824 66,674 65,858 \n4.

In [49]:
cosine_similarity(embeddings.embed_query("Give me docs about stats"), embeddings.embed_query("[04637] 서울특별시 중구 퇴계로 10, 메트로타워   \nT 02-6908-8200   F 02-6312-8959   H www.kpfis.or.kr\nA Guide to Fiscal Statistics\n재정통계 해설')"))

0.44784402320714384

In [51]:
np.dot(embeddings.embed_query("Give me docs about stats"), embeddings.embed_query("[04637] 서울특별시 중구 퇴계로 10, 메트로타워   \nT 02-6908-8200   F 02-6312-8959   H www.kpfis.or.kr\nA Guide to Fiscal Statistics\n재정통계 해설')"))

7.775198208304589

In [22]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "Bllossom/llama-3.2-Korean-Bllossom-3B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    torch_dtype = "bfloat16",
    device_map = 'auto',
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.3,
    top_k=5,
    top_p=0.95,
    return_full_text=False,
    max_new_tokens=128,

)
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [24]:
({"context": retriever | format_docs, "question": RunnablePassthrough()}| prompt | llm).invoke(question)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


' 2024년 중앙정부의 예산 지출은 다음과 같이 구성되어 있습니다.\n     1. 인건비( Personnel Expenses )\n     2. 소비자물품비( Consumption of Goods and Services )\n     3. 재정비( Financial Expenses )\n     4. 재정수입비( Financial Income )\n     5. 기타비( Other Expenses )\n     6. 재정수입( Financial Income )\n     7. 재정비( Financial Expenses )\n     8. 소비자물품비( Consumption of Goods and Services )\n     9. 인건비('

In [9]:
df = pd.read_csv(path[:-7]+".csv")

In [10]:
documents={}
retrievers={}
for i in range(len(pdfs)):
    documents[pdfs[i]] = FAISS.from_documents([doc for doc in doc_texts if doc.metadata["source"] == pdfs[i]], embedding=embeddings)
    retrievers[pdfs[i]] = documents[pdfs[i]].as_retriever(search_type="mmr", search_kwargs={'k': 3, 'fetch_k': 8})

In [11]:
print(documents[pdfs[0]])
print(retrievers[pdfs[0]])

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f5741f551e0> search_type='mmr' search_kwargs={'k': 3, 'fetch_k': 8}


In [12]:
df

,SAMPLE_ID,Source,Source_path,Question,Answer
0,TRAIN_000,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?,"2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년..."
1,TRAIN_001,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?,"2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7..."
2,TRAIN_002,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,기금이 예산과 다른 점은?,"기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정..."
3,TRAIN_003,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"일반회계, 특별회계, 기금 간의 차이점은 무엇인가요?","일반회계는 특정 사업 운영 및 특정 세입으로 특정 세출을 충당하는데 사용되고, 특별..."
4,TRAIN_004,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"2024년 총수입은 얼마이며, 예산수입과 기금수입은 각각 몇 조원인가요?","2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216..."
...,...,...,...,...,...
491,TRAIN_491,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,자치단체 보조금과 민간보조금은 각각 어떤 비율로 증가했는가?,"자치단체 보조금은 2019년 대비 2022년에 35% 증가하였고, 민간보조금은 10..."
492,TRAIN_492,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,한국의 재정금융안정계획이 주로 어떤 목표에 초점을 맞추어 있었는가?,한국의 재정금융안정계획은 통화량 조절과 물가안정이라는 단기적 목표에 초점을 맞추어 ...
493,TRAIN_493,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,1952년에 체결된 '한미경제조정협정'은 어떤 문제를 해결하기 위해 체결되었는가?,"원조물자 판매대금의 효과적 활용, 참전유엔군 경비지출을 위해 우리 정부에서 대여해 ..."
494,TRAIN_494,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,"2023년 12월 IMF는 성장 전망을 어떻게 제시하고 있으며, 성장세의 둔화가 어...","IMF는 최근 세계 경제전망을 통해 작년 3.5%에서 올해 3%, 내년 2.9%로 ..."


In [13]:
def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

In [14]:
({"context": retriever | format_docs, "question": RunnablePassthrough()}| prompt | llm).invoke(question)

NameError: name 'retriever' is not defined

In [15]:
# DataFrame의 각 행에 대해 처리
results = []    
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    # source = normalize_string(row['Source'])  #이런 잡스러운건 전부 주석으로
    source = (row['Source'])
    source = os.path.join(path, source+".pdf",)
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    # normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    # normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    # normalized_keys = {(k): v for k, v in db.items()}
    # normalized_keys = {(k): v for k, v in db.items()}

    retriever = retrievers[source]

    # RAG 체인 구성
    template = """
    다음 정보를 바탕으로 질문에 답하세요:
    {context}

    질문: {question}

    답변:
    """
    prompt = PromptTemplate.from_template(template)

    # RAG 체인 정의
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)

    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })

Answering Questions:   0%|          | 0/496 [00:00<?, ?it/s]

Question: 2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Answering Questions:   0%|          | 1/496 [00:11<1:34:43, 11.48s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Answer:  2024년 중앙정부 재정체계는 다음과 같이 구성되어 있습니다.
     1. 재정체계의 구성: 
        - 재정체계는 정부의 재정적 인적, 재정적 인적, 재정적 인적, 재정적 인적, 재정적 인적, 재정적 인적, 재정적 인적, 재정적 인적, 재정적 인적, 재정적 인적, 재정적 인적, 재정적 인적, 재정적 인적, 재정적 인적, 재정적 인적,

Question: 2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?


Answering Questions:   0%|          | 1/496 [00:14<1:56:45, 14.15s/it]


KeyboardInterrupt: 